# This is the example flow for the MINT transformation pipeline

This notebook abstracts an example flow for a transformation flow in the MINT project, we show a simple CSV --> CSV file based on user input

## Part 1: The Reader Adapter

In this stage we have a `input.csv` file and a `input.yaml` file describing the D-REPR layout, we create a general representation of the data in a python object (or a materialized representation of that object in a file)

In [1]:
# {input.*, input.yaml} --> Graph (Obj)

## Part 2: The Transformation Library

In this stage we have a python object (or a materialized representation of that object in a file) representing the data in a general fashion

In [ ]:
# Graph(Obj) --> Graph(Obj)

## Part 3: The Writer Adapter

In this stage we want to output a materialized output file (`output.csv`) based on a given `output.yaml` and an additional configuration file `output.config`

In [2]:
# {Graph(Obj), output.yaml} --> {output.*}

## This is the semantic description of the interface between all of the components in the flow (Reader/Writer Adapters, Transformation-Steps)

In [1]:
enum ArgType:
	FilePath(Format),
	Graph(SemanticModel),
 	NumpyArray(Schema),
	String,
	Number,
	Boolean,
	DateTime


class TransFunc:
	id: Str
	inputs: Dict[str, Optional[ArgType]];
	output: Dict[str, Optional[ArgType]];

	def __init__(self, config, data, target_unit, source_unit):
		pass

	def validate(self) -> bool:
		pass

	def exec(self) -> dict:
		pass
		

class UnitTransformation(TransFunc):
	id: Str = "unit_transformation"
	description: Str = ""
	inputs = {
		"conf": ArgType.FilePath,
		"source_unit": ArgType.String,
		"data": ArgType.Graph
	}
	outputs = {
		"data": ArgType.Graph
	}


	def __init__(self, config, data, target_unit, source_unit):
		pass

	def validate(self) -> bool:
		pass

	def exec(self) -> dict:
		pass

pipeline = [
	ReaderPDF,
	UnitTransformation
]

args = {
	"<id>_<index>_<arg_name>": <value>	
}
args['unit_transformation_1_arg_name'] = None
pipeline.exec(args)

SyntaxError: invalid syntax (<ipython-input-1-2d9e2d75fa9d>, line 1)